In [10]:
import torch

# --- 1. THE DICTIONARY ---
state_map = {0: "Rainy", 1: "Sunny"}
obs_map = {0: "Walk", 1: "Shop", 2: "Clean"}

# --- 2. THE MODEL PARAMETERS ---

# TRANSITION PROBABILITIES (trans)
# Row 0: Rainy -> [Rainy, Sunny]
# Row 1: Sunny -> [Rainy, Sunny]
# TPM : Transition Probability Matrix
trans = torch.tensor([
    [0.7, 0.3],
    [0.4, 0.6]
])

# EMISSION PROBABILITIES (emit)
# Row 0 (Rainy) -> [Walk, Shop, Clean]
# Row 1 (Sunny) -> [Walk, Shop, Clean]
emit = torch.tensor([
    [0.1, 0.4, 0.5], 
    [0.6, 0.3, 0.1]   
])

# INITIAL PROBABILITIES (start_probs)
# Our "Day 1 Forecast" before seeing Bahadur.
# We believe there is a 60% chance it starts Rainy, 40% Sunny.
start_probs = torch.tensor([0.6, 0.4])

# OBSERVATION SEQUENCE (obs)
# Bahadur's Diary: Walk (0), Shop (1), Clean (2)
obs = torch.tensor([0, 1, 2]) 



In [11]:
# trans.shape -> array
# [0] -> gives first element
N = trans.shape[0]


In [8]:
print (N)

2


In [16]:

# --- 3. THE ALGORITHMS ---

def forward_algorithm(trans, emit, start_probs, obs):
    """
    Calculates the likelihood of the observation sequence.
    """
    N = trans.shape[0] # Number of states
    T = len(obs)
    alpha = torch.zeros(T, N)

    # Initialization (Day 1)
    # "Prior belief" combined with "First evidence"
    alpha[0] = start_probs * emit[:, obs[0]]

    # Recursion (Day 2 to T)
    for t in range(1, T):
        # for every observation
        for j in range(N): # j goes from 0 to 1 inclusive (because there are two states)
            # How likely is it we arrived at state j from ANY previous state?
            prob_transition = torch.sum(alpha[t-1] * trans[:, j])
            
            # Update with the likelihood of today's evidence
            alpha[t, j] = prob_transition * emit[j, obs[t]]

    return alpha

def viterbi_algorithm(trans, emit, start_probs, obs):
    """
    Decodes the most likely weather sequence.
    """
    N = trans.shape[0]
    T = len(obs)
    
    delta = torch.zeros(T, N)
    psi = torch.zeros(T, N, dtype=torch.long)

    # Initialization (Day 1)
    delta[0] = start_probs * emit[:, obs[0]]

    # Recursion
    for t in range(1, T):
        for j in range(N):
            # Calculate path prob from every previous state i to state j
            paths = delta[t-1] * trans[:, j] * emit[j, obs[t]]
            
            # Save the max prob and the state that gave us that max
            delta[t, j] = torch.max(paths)
            
            print("Max probs: ", torch.max(paths))
            psi[t, j] = torch.argmax(paths)
            print("State: ",torch.argmax(paths))

    # Backtracking
    best_path = torch.zeros(T, dtype=torch.long)
    best_path[-1] = torch.argmax(delta[-1])
    
    for t in range(T-2, -1, -1):
        best_path[t] = psi[t+1, best_path[t+1]]
        
    return best_path, torch.max(delta[-1])



In [17]:
# --- 4. THE DEMO ---

print(f"Bahadur's Diary: {[obs_map[i.item()] for i in obs]}\n")

# Run Forward
alpha = forward_algorithm(trans, emit, start_probs, obs)
print(alpha.shape)
print(alpha)
total_prob = torch.sum(alpha[-1])

print(f"Likelihood of this diary: {total_prob:.4f} (approx {total_prob*100:.1f}%)")


Bahadur's Diary: ['Walk', 'Shop', 'Clean']

torch.Size([3, 2])
tensor([[0.0600, 0.2400],
        [0.0552, 0.0486],
        [0.0290, 0.0046]])
Likelihood of this diary: 0.0336 (approx 3.4%)


In [18]:

# Run Viterbi
best_path, best_prob = viterbi_algorithm(trans, emit, start_probs, obs)

print("\n--- Cellar occupant's Conclusion (Viterbi) ---")
for t, state_idx in enumerate(best_path):
    day_num = t + 1
    activity = obs_map[obs[t].item()]
    weather = state_map[state_idx.item()]
    print(f"Day {day_num}: Bahadur '{activity}' -> Weather was likely {weather}")

Max probs:  tensor(0.0384)
State:  tensor(1)
Max probs:  tensor(0.0432)
State:  tensor(1)
Max probs:  tensor(0.0134)
State:  tensor(0)
Max probs:  tensor(0.0026)
State:  tensor(1)

--- Cellar occupant's Conclusion (Viterbi) ---
Day 1: Bahadur 'Walk' -> Weather was likely Sunny
Day 2: Bahadur 'Shop' -> Weather was likely Rainy
Day 3: Bahadur 'Clean' -> Weather was likely Rainy
